In [1]:
import re
import os

%pip install pandas
import pandas as pd

%pip install --upgrade pip

%pip install requests
import requests

%pip install bs4
from bs4 import BeautifulSoup

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
#setting working directory to eml_files
os.chdir('eml_files')

In [4]:
def urls_in_file(file_name):
    """ Takes an .eml file name as a string and returns a list of all unique urls linking to news sources """

    #reads the file and removes newlines and equal signs (which are placed at the end of each line)
    with open(file_name, 'r') as file:
        textfile = file.read().replace('\n', '').replace('=', '')
    
    #any strings without white spaces starting with 3Dhttps:// and ending with &amp, non-greedy matching
    valid_urls = re.findall('3D(https://\S*?)?&amp', textfile)
    
    #creates a list of each unique url (removes duplicates)
    unique_urls = list(set(valid_urls))
    
    return unique_urls

#demonstrates the function with the first file in the folder
urls_in_file('1. El Salvador Mineria 1-94 eml/Google Alert - El Salvador minería_1.eml')

['https://diario.elmundo.sv/nacionales/organizaciones-dicen-que-el-sistema-judicial-esta-a-prueba-tras-la-entrega-de-mas-de-59000-firmas-contra-de-la-mineria',
 'https://www.elsalvador.com/opinion/editoriales/mineria-/1209390/2025/',
 'https://www.laprensagrafica.com/tendencias/Un-Pikachu-revolucionario-un-manifestante-disfrazado-de-Pokemon-se-unio-a-las-protestas-en-Turquia-20250329-0020.html',
 'https://www.elsalvador.com/noticias/nacional/mercados-mercado-cuscatlan-alcaldia-de-san-salvador-centro-/1209302/2025/',
 'https://www.laprensagrafica.com/opinion/Las-locuras-del-emperador-II-20250328-0081.html',
 'https://www.revistafactum.com/slm-grok/',
 'https://diario.elmundo.sv/el-mundo/un-pueblo-del-sur-de-honduras-podria-perder-su-historica-iglesia-por-la-explotacion-minera']

In [5]:
def urls_in_folder(folder_name):
    """ Takes a folder of .eml files as a string and returns a list of all unique urls contained in each of the 
    files in the folder 
"""
    
    #creates a list of all file names in folder_name
    filenames = os.listdir(folder_name)
    filenames = [os.path.join(folder_name, file) for file in filenames]
    
    #creates a list to store the unique urls found in each file using urls_in_file function
    all_urls = []
    for file in filenames:
        urls = urls_in_file(file)
        all_urls.extend(urls)
    
    return all_urls

#demonstrates the function with folder '1. El Salvador Mineria 1-94 eml'
urls_in_folder('1. El Salvador Mineria 1-94 eml')[:5]

['https://www.laprensagrafica.com/elsalvador/El-Chino-Flores-secretario-del-FMLN-dice-que-reactivara-al-partido-20250107-0097.html',
 'https://diario1.com/zona-deportiva/2025/01/el-real-madrid-se-regala-una-goleada-en-cartagena-antes-de-la-supercopa/',
 'https://lapagina.com.sv/deportes/ancelotti-se-lleva-a-arabia-a-cuatro-canteranos-y-a-alaba-para-buscar-el-tercer-titulo-de-la-temporada/',
 'https://www.elsalvador.com/opinion/editoriales/mineria-/1192178/2025/',
 'https://www.contrapunto.com.sv/muro-colapso-tras-sismo-en-san-salvador-y-dejo-a-una-persona-herida/']

In [17]:
def extract_website_text(url):
    """Takes in a url as a string and returns the text content of the webpage, including headers, footers, and paragraphs."""
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        extracted_text = soup.get_text(separator='\n', strip=True)
        return extracted_text
    except requests.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return None
    

def clean_extracted_text(text, section='body', cutoff=20):
    """Cleans the text extracted using extract_website_text and removes short paragraphs (headers, footers, etc...) based on word count cutoff"""
    if text is None:
        return None
    elif section == 'body':
        split = text.split('\n') #split the text into a list divided every time there is a new line \n
        body = [paragraph for paragraph in split if len(paragraph.split()) > cutoff] #remove paragraphs with less than CUTOFF words.
        body_text = ' '.join(body) #combine remaining text into a single string
        return body_text
    elif section == 'header':
        # Assuming the header is the first part of the text before the first paragraph
        header = text.split('\n')[0] if '\n' in text else text.split('-')[0]
        return header.strip()


example_text = extract_website_text('https://www.contrapunto.com.sv/muro-colapso-tras-sismo-en-san-salvador-y-dejo-a-una-persona-herida/')
clean_extracted_text(example_text, section='body')

'Una persona quedó atrapada tras el colapso de un muro en San Salvador, provocado por un sismo. Fue rescatada y atendida por Cruz Verde Salvadoreña. Una persona quedó atrapada bajo los escombros de un muro que colapsó en la colonia Las Flores, calle Agua Caliente, San Salvador Este, tras el sismo registrado la tarde del lunes 5 de enero a las 5:37 p. m.; La víctima, cuya identidad no ha sido revelada, fue localizada por un residente del lugar y posteriormente rescatada por elementos de Cruz Verde Salvadoreña, quienes le brindaron primeros auxilios antes de trasladarla a un centro asistencial. Este sismo fue una de las réplicas del movimiento telúrico de magnitud 6.3 que ocurrió frente a la costa de La Paz a las 11:18 a.m. del domingo 5 de enero. Según el Ministerio de Medio Ambiente, desde entonces se han registrado 147 réplicas, de las cuales 21 han sido sentidas por la población. Las autoridades explicaron que esta actividad sísmica es producto del proceso de subducción entre las pla

In [18]:
def extract_info_from_emls_in_folder(folder_name):
    """Extracts text from all .eml files in a folder and returns a table filled with urls, article titles, and body text."""
    
    urls = urls_in_folder(folder_name)
    
    raw_url_text = [extract_website_text(url) for url in urls]
    cleaned_text = [clean_extracted_text(text, section='body') for text in raw_url_text]
    extracted_info = pd.DataFrame({
        'url': urls_in_folder,
        'title': [clean_extracted_text(text, section='header') for text in raw_url_text],
        'body': cleaned_text
    })
    
    return extracted_info

#mineria_1 = extract_info_from_emls_in_folder('1. El Salvador Mineria 1-94 eml')
#mineria_1

In [19]:
def emls_in_file_to_table(file_path):
    """Extracts text from the file_path in a folder and returns a table filled with urls, article titles, and body text."""
    
    urls = [url for url in urls_in_file(file_path)]
    
    raw_url_text = [extract_website_text(url) for url in urls]
    cleaned_text = [clean_extracted_text(text, section='body') for text in raw_url_text]
    extracted_info = pd.DataFrame({
        'url': urls,
        'title': [clean_extracted_text(text, section='header') for text in raw_url_text],
        'body': [clean_extracted_text(text, section='body') for text in raw_url_text]
    })
    
    return extracted_info

mineria_2 = emls_in_file_to_table('1. El Salvador Mineria 1-94 eml/Google Alert - El Salvador minería_1.eml')
mineria_2

Error fetching https://www.laprensagrafica.com/tendencias/Un-Pikachu-revolucionario-un-manifestante-disfrazado-de-Pokemon-se-unio-a-las-protestas-en-Turquia-20250329-0020.html: 403 Client Error: Forbidden for url: https://www.laprensagrafica.com/tendencias/Un-Pikachu-revolucionario-un-manifestante-disfrazado-de-Pokemon-se-unio-a-las-protestas-en-Turquia-20250329-0020.html
Error fetching https://www.laprensagrafica.com/opinion/Las-locuras-del-emperador-II-20250328-0081.html: 403 Client Error: Forbidden for url: https://www.laprensagrafica.com/opinion/Las-locuras-del-emperador-II-20250328-0081.html


,url,title,body
0,https://diario.elmundo.sv/nacionales/organizac...,Organizaciones dicen que el sistema judicial e...,Organizaciones dicen que el sistema judicial e...
1,https://www.elsalvador.com/opinion/editoriales...,¿Por qué se involucran las iglesias en el tema...,"El periodismo que hacemos requiere tiempo, esf..."
2,https://www.laprensagrafica.com/tendencias/Un-...,None,None
3,https://www.elsalvador.com/noticias/nacional/m...,El mercado Cuscatlán funciona apenas al 40% de...,"El periodismo que hacemos requiere tiempo, esf..."
4,https://www.laprensagrafica.com/opinion/Las-lo...,None,None
5,https://www.revistafactum.com/slm-grok/,La obsesión de Bukele con la minería y con su ...,"En el episodio 9 de Sobre La Mesa, el podcast ..."
6,https://diario.elmundo.sv/el-mundo/un-pueblo-d...,Un pueblo del sur de Honduras podría perder su...,La minera artesanal opera en el barrio El Cent...
